In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import seaborn as sns
def corr_heatmap(df):
    corr = df.corr()
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style(style="ticks"):
        f, ax = plt.subplots(figsize=(9, 6))
    ax = sns.heatmap(corr, mask=mask, annot=True, linewidths=1, cmap="YlGnBu")

In [ ]:
def macro_accuracy(actual, pred):
    assert len(actual) == len(pred)
    correct = 0
    for i, j in zip(actual, pred):
        if i == j:
            correct += 1
    return correct / len(actual)
    